In [ ]:
-- %defaultDatasource jdbc:mysql://docker.for.mac.localhost/ckb_utilities?user=ckb_utilities&password=ckb_utilities&useSSL=false&serverTimezone=UTC

%defaultDatasource jdbc:mysql://localhost/ckb_utilities?user=ckb_utilities&password=ckb_utilities&useSSL=false&serverTimezone=UTC

select gv.protein_effect, count(*)
into ${proteinEffectGroupByRs}
from gene g
    join gene_variant gv on gv.gene_id = g.gene_id
where gv.protein_effect !=''
group by gv.protein_effect
order by gv.protein_effect desc

In [ ]:
%%groovy
// println(beakerx.proteinEffectGroupByRs.values)
// def map = beakerx.proteinEffectGroupByRs.values.collectEntries{[(it[0]?:'not applicable'):it[1]]}

def map = ['unknown': 19649, 'no effect - predicted': 1014, 'no effect': 1335, 'loss of function - predicted': 3222, 'loss of function': 1833, 'gain of function - predicted': 818, 'gain of function': 1370]
beakerx.proteinEffectCategories = map.collect{it.key}
beakerx.proteinEffectCounts = map.collect{it.value}

beakerx.barTraces = map.collect{
    [
       x: ['Effect'] ,
       y: [it.value],
       type: 'bar',
       name : it.key, 
    ]
}

println( map.collect{"'${it.key}': ${it.value}"} )



Visuals

In [ ]:
%%javascript
require.config({paths:{'plotly':'//cdn.plot.ly/plotly-latest.min'}});
require(['plotly'], function(Plotly) {
    window.Plotly = Plotly;
});


In [ ]:
%%javascript

var pieData = [{
  values: beakerx.proteinEffectCounts,
  labels: beakerx.proteinEffectCategories,
  type: 'pie',
  name : 'Effect',
  hoverinfo: 'label+percent+value',
  hole: .6,
  sort:false
}];

var layout = {
    title: 'Protein Effect',
    height: 1000,
    width: 1000,
    showlegend: true,
    legend : {
        itemclick: false,
        itemdoubleclick: false,
        orientation: 'v',
        y: 1,
        traceorder: 'reversed',
    }, 
    barmode: 'stack',
}


beakerx.displayHTML(this, '<div id="plotlyDonutEffect"/>');
Plotly.newPlot('plotlyDonutEffect', pieData, layout);

// beakerx.displayHTML(this, '<div id="plotlyBarEffect"/>');
//  Plotly.newPlot('plotlyBarEffect', beakerx.barTraces, layout);